# OTSU Big Data Cloud notebook

In [1]:
import gcsfs
import rioxarray
import rasterio
import os
import numpy as np
from distributed import Client
import xarray as xr

## Accès à un bucket GCP

In [2]:
import gcsfs
fs = gcsfs.GCSFileSystem(bucket_name="supaero", token='anon')

In [3]:
fs.ls('supaero/31TCH')

['supaero/31TCH/SENTINEL2A_20190804-105917-289_L2A_T31TCH_C_V2-2',
 'supaero/31TCH/SENTINEL2A_20191003-105914-893_L2A_T31TCH_C_V2-2',
 'supaero/31TCH/SENTINEL2A_20200210-105905-458_L2A_T31TCH_C_V2-2',
 'supaero/31TCH/SENTINEL2A_20200318-104912-044_L2A_T31TCH_C_V2-2',
 'supaero/31TCH/SENTINEL2B_20190401-105915-368_L2A_T31TCH_C_V2-0',
 'supaero/31TCH/SENTINEL2B_20190528-104924-205_L2A_T31TCH_C_V2-1',
 'supaero/31TCH/SENTINEL2B_20190630-105921-483_L2A_T31TCH_C_V2-2',
 'supaero/31TCH/SENTINEL2B_20190720-105921-577_L2A_T31TCH_C_V2-2',
 'supaero/31TCH/SENTINEL2B_20191117-105911-155_L2A_T31TCH_C_V2-2',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2']

In [4]:
fs.ls('supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2')

['supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/DATA',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/MASKS',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_ATB_R1.tif',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_ATB_R2.tif',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B11.tif',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B12.tif',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B2.tif',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B3.tif',
 'supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910

## Lecture d'un produit, affichage d'une bande

In [5]:
with rasterio.Env(GS_NO_SIGN_REQUEST="YES"):
    xds_11 = rioxarray.open_rasterio("gs://supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B11.tif", nodata=-10000.0)
    xds_11 = xds_11.where(xds_11 != -10000)
    xds_11.rio.write_nodata(-10000, encoded=True, inplace=True)

xds_11

<xarray.DataArray (band: 1, y: 5490, x: 5490)> Size: 121MB
array([[[  nan,   nan,   nan, ..., 1609., 1549., 1496.],
        [  nan,   nan,   nan, ..., 1655., 1682., 1605.],
        [  nan,   nan,   nan, ..., 1738., 1782., 1642.],
        ...,
        [  nan,   nan,   nan, ...,  644.,  538.,  516.],
        [  nan,   nan,   nan, ...,  693.,  673.,  556.],
        [  nan,   nan,   nan, ...,  495.,  606.,  546.]]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 44kB 3e+05 3e+05 3e+05 ... 4.098e+05 4.098e+05
  * y            (y) float64 44kB 4.8e+06 4.8e+06 4.8e+06 ... 4.69e+06 4.69e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [ ]:
xds_11.plot()

## Construction d'un Dataset monotemporel